# **I. DATA PRE-PROCESSING**

In this section, we perform the following tasks: 
- Import Python libraries crucial to the optimization process, including **pandas**, **numpy**, and **cvxpy**.
- Read the provided *workers.csv* file and extract the Preferences Matrix (**P**), Skills Matrix (**S**), and Teammate [Dis-]Preferences Matrices (**MD** and **MP**). For more information about these matrices, please check our optimization project report.
- Declare Optimization Variable **X** of dimension (20, 4) with binary elements representing whether or not a worker is in a project
- Declare all constraints as instructed in the provided text file (10 constraints in total) and a shared constraint specifying the sum-of-1 nature for each row in X



In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp

df = pd.read_csv('workers.csv', sep=',')
df.fillna('', inplace=True)
df

,Name,Optimization,Organization,Teamwork,Project 1,Project 2,Project 3,Project 4,Preferred Partners,Dispreferred Partners
0,Ash,10,5,6,-2,1,-2,4,"Caitlin,Flint,Jasmine,Misty,Pryce,Whitney","Dahlia,Hala,Kabu,Lt. Surge"
1,Brock,5,8,2,-1,1,-4,4,"Dahlia,Rosa,Sabrina,Valerie,Whitney",Flint
2,Caitlin,7,8,6,3,2,2,-2,"Ash,Dahlia,Nate,Pryce,Valerie","Gary,Kabu,Lt. Surge,Misty,Rosa"
3,Dahlia,1,1,9,-1,-4,-2,-5,"Caitlin,Iris,Misty","Ash,Oak"
4,Elaine,2,8,8,-3,3,-5,5,"Caitlin,Jasmine,Lt. Surge,Misty,Nate,Oak,Rosa,...","Kabu,Pryce"
5,Flint,4,10,5,-4,4,-5,-4,"Ash,Gary,Jasmine,Misty,Nate,Sabrina,Valerie","Brock,Pryce"
6,Gary,7,1,3,-2,-1,-1,0,"Flint,Hala,Kabu,Lt. Surge,Sabrina,Valerie,Whitney",Caitlin
7,Hala,6,4,9,3,1,0,-2,"Caitlin,Gary,Iris,Jasmine,Oak,Pryce,Rosa,Whitney","Ash,Misty"
8,Iris,1,1,10,-2,-3,1,-5,"Caitlin,Hala,Misty,Oak,Valerie",Flint
9,Jasmine,4,4,9,-4,0,-3,2,"Ash,Elaine,Flint,Iris,Misty,Nate",Oak


In [ ]:
# DATA PREPROCESSING
# Preferences Matrix
P_df = pd.DataFrame(data=df[['Project 1','Project 2','Project 3','Project 4']].values, index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8)
# Skills Matrix
S_df = pd.DataFrame(data=df[['Optimization','Organization','Teamwork']].values, index=df['Name'].values, columns=['Optimization','Organization','Teamwork'], dtype=np.int8)

In [ ]:
# Shift range of preferences score from (-5,5) to (0, 10)
P_df = P_df.transform(lambda x: x+5)
P_df

,Project 1,Project 2,Project 3,Project 4
Ash,3,6,3,9
Brock,4,6,1,9
Caitlin,8,7,7,3
Dahlia,4,1,3,0
Elaine,2,8,0,10
Flint,1,9,0,1
Gary,3,4,4,5
Hala,8,6,5,3
Iris,3,2,6,0
Jasmine,1,5,2,7


In [ ]:
P = P_df.values # dim = (20,4)
S = S_df.values # dim = (20,3)
X = cp.Variable((20,4), boolean=True) # dim = (20,4), corresponds to whether person ith is in project jth

In [ ]:
# SHARED CONSTRAINT
constraints_shared = [cp.sum(X[i]) == 1 for i in range(20)]

In [ ]:
# CONSTRAINT PROJECT 1 (write as a list then we can concat them later) (Danny)
total_members_1 = cp.sum(X[:,0]) # At least 4 team members
highest_organization_1 = np.max(S[:,1] * X[:,0]) # At least one team member needs an Organization skill of at least 7
each_teamwork_1 = cp.multiply(S[:,2], X[:,0]) # Teamwork skill must be at least 4 for each team member
teamwork_at_least_4 = X[:,0] * 4 

constraints_1 = [total_members_1 >= 4,
                 highest_organization_1 >= 7,
                 each_teamwork_1 >= teamwork_at_least_4]

In [ ]:
# CONSTRAINT PROJECT 2 (write as a list then we can concat them later) (Lanxuan)
total_members_2 = cp.sum(X[:,1]) # At least 4 team members
avg_each_teamwork_2 = cp.sum(S[:,2] * X[:,1]) / cp.sum(X[:,1]) # Average Teamwork skill of team members must be at least 5

constraints_2 = [total_members_2 >= 4,
                 avg_each_teamwork_2 >= 5]

In [ ]:
# CONSTRAINT PROJECT 3 (write as a list then we can concat them later) (Jian)
total_members_3 = cp.sum(X[:,2]) # Between 3 and 6 team members
total_organization_3 = S[:,1].T @ X[:,2] # Total Organization skill among team members must be at least 20
total_optimization_3 = S[:,0].T @ X[:,2] # Total Optimization skill among team members must be at least 12

constraints_3 = [total_members_3 >= 3, 
                 total_members_3 <= 6, 
                 total_organization_3 >= 20, 
                 total_optimization_3 >= 12]

In [ ]:
# CONSTRAINT PROJECT 4 (write as a list then we can concat them later) (Vy)
total_members_4 = cp.sum(X[:,3]) # At least 3 team members
each_optimization_4 = cp.multiply(S[:,0], X[:,3]) # Optimization skill must be at least 4 for each team member
optimization_at_least_4 = X[:,3] * 4 

constraints_4 = [total_members_4 >= 3,
                 each_optimization_4 >= optimization_at_least_4]

In [ ]:
constraints = constraints_shared + constraints_1 + constraints_2 + constraints_3 + constraints_4

We create 2 binary matrices (0,1) based on each worker's teammate preferences and dispreferences: 
- MP: Teammate Preferences Matrix, where MP[i][j] = 1 meaning worker ith prefers worker jth to be a teammate, else = 0
- MD: Teammate Dispreferences Matrix, where MD[i][j] = 1 meaning worker ith does not prefer worker jth to be a teammate, else = 0

In [ ]:
MP_df = pd.DataFrame(data=np.zeros((20,20), dtype=np.int8), columns=df['Name'].values, index=df['Name'].values) # Teammate Preferences Matrix
MD_df = pd.DataFrame(data=np.zeros((20,20), dtype=np.int8), columns=df['Name'].values, index=df['Name'].values) # Teammate Dispreferences Matrix

for _, this_person_data in df.iterrows():
  preferred = this_person_data['Preferred Partners'].split(",")
  for name in preferred:
    if name in df['Name'].values:
      MP_df[name][this_person_data['Name']] = 1
  
  dispreferred = this_person_data['Dispreferred Partners'].split(",")
  for name in dispreferred:
    if name in df['Name'].values:
      MD_df[name][this_person_data['Name']] = 1

In [ ]:
MP_df

,Ash,Brock,Caitlin,Dahlia,Elaine,Flint,Gary,Hala,Iris,Jasmine,Kabu,Lt. Surge,Misty,Nate,Oak,Pryce,Rosa,Sabrina,Valerie,Whitney
Ash,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1
Brock,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
Caitlin,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
Dahlia,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
Elaine,0,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,1,0,1,1
Flint,1,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,1,1,0
Gary,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,1,1,1
Hala,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1
Iris,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0
Jasmine,1,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0


In [ ]:
MD_df

,Ash,Brock,Caitlin,Dahlia,Elaine,Flint,Gary,Hala,Iris,Jasmine,Kabu,Lt. Surge,Misty,Nate,Oak,Pryce,Rosa,Sabrina,Valerie,Whitney
Ash,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0
Brock,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Caitlin,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0
Dahlia,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Elaine,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
Flint,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
Gary,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Hala,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
Iris,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Jasmine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
MAX_DISPREFERRED = np.max(MD_df.values.sum(axis=1)) # it should be 5 (maximum number of dispreferred teammaters out of all workers)

def max_number_of_dispreferred(n):
  return np.full((20,4), MAX_DISPREFERRED) - X * (MAX_DISPREFERRED - n)

# **II. MAXIMIZE PREFERENCES**

This section presents the first optimization approach: maximizing workers' preferences in all projects. In other words, we are maximizing the sum of total preferences score for project 1, project 2, project 3, and project 4.

We also present three differences approaches in modifying the constraints: the provided constraints, enhancement with teammate dispreferences, and enhancement with teammate preferences.

*For details regarding the optimization problem's formulation and motivation, please check our optimization project report*

## **1. Straightforward attempt:** 

We use the provided constraints. 

### **a. Primal Form:**

STATUS: optimal

In [ ]:
# DECLARE MAIN OBJECTIVE FUNCTION
obj_function = cp.sum(cp.multiply(P, X))
objective = cp.Maximize(obj_function)

In [ ]:
# CALL OPTIMIZATION AND GET RESULT
problem = cp.Problem(objective, constraints)
try:
  problem.solve(qcp=True)
except KeyError:
  problem.solve(qcp=True, solver=cp.ECOS_BB)

print("status:", problem.status)
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", objective.value)
  print("resulting matrix:\n", X.value)
  print("formatted:")

formatted_X = pd.DataFrame(np.round(np.abs(X.value), 0), index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8) if X.value is not None else None
formatted_X

status: optimal
optimal value: 154.0
resulting matrix:
 [[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
formatted:


,Project 1,Project 2,Project 3,Project 4
Ash,0,0,0,1
Brock,0,0,0,1
Caitlin,1,0,0,0
Dahlia,1,0,0,0
Elaine,0,1,0,0
Flint,0,1,0,0
Gary,0,0,0,1
Hala,1,0,0,0
Iris,0,0,1,0
Jasmine,0,0,0,1


### **b. Dual Form:**

STATUS: OPTIMAL.

We found a certificate of optimality with an optimal value of 154. 

*There are floating point errors in the CVXPY library's output of the dual optimal value and optimal solution which are fixed using ceil/round functions. Nonetheless, it is sufficient to certify the primal's optimal solution.*

In [ ]:
# DECLARE DUAL VARIABLES
y = cp.Variable(20, name="y")
w = cp.Variable(20, name="w")
q = cp.Variable(20, name="q")
r = cp.Variable(20, name="r")
a = cp.Variable(1, name="a")
b = cp.Variable(1, name="b")
c = cp.Variable(1, name="c")
d = cp.Variable(1, name="d")
e = cp.Variable(1, name="e")
f = cp.Variable(1, name="f")
z = cp.Variable(1, name="z")
n = cp.Variable(1, name="n")
# based on optimal solution (binary variable defining the worker's state of being assigned to project 1 with organization skill >= 7)
# more details in the report
k = np.zeros(20)
k[2], k[17] = 1, 1 

In [ ]:
# DECLARE DUAL CONSTRAINTS
constraints_dual = []
constraints_dual.extend([y[i] + z + S[i,1]*w[i] + (S[i,2] - 4)*q[i] >= P[i,0] for i in range(20)])
constraints_dual.extend([y[i] + a + (S[i,2] - 5)*n >= P[i,1] for i in range(20)])
constraints_dual.extend([y[i] + b + c + S[i,1]*e + S[i,0]*f >= P[i,2] for i in range(20)])
constraints_dual.extend([y[i] + d + (S[i,0] - 4)*r[i] >= P[i,3] for i in range(20)])
constraints_dual.extend([r[i] <= 0 for i in range(20)])
constraints_dual.extend([q[i] <= 0 for i in range(20)])
constraints_dual.extend([w[i] <= 0 for i in range(20)])
constraints_dual.extend([c >= 0, a <= 0, b <= 0, d <= 0, e <= 0, f <= 0, z <= 0])

In [ ]:
# DECLARE DUAL OBJECTIVE FUNCTION
dual_objective_function = cp.sum(y) + cp.sum([7*k[i]*w[i] for i in range(20)]) + 20*e + 12*f + 4*z + 4*a + 3*b + 6*c + 3*d
dual_objective = cp.Minimize(dual_objective_function)

In [ ]:
# CALL OPTIMIZATION AND GET RESULT
dual_problem = cp.Problem(dual_objective, constraints_dual)
dual_problem.solve()

print("status:", dual_problem.status)
if dual_problem.status == cp.OPTIMAL or dual_problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", np.ceil(dual_objective.value))
  for var in dual_problem.variables():
    print(var.name(), ':', var.value.round(5))

status: optimal
optimal value: 154.0
y : [ 9.    9.    8.    4.    7.75  9.5   5.    8.    6.    7.    3.   10.
  6.   10.   10.    9.    3.   10.   10.   11.25]
w : [-0.44945 -0.67081 -0.       0.      -0.26389 -0.37806 -2.34561  0.
 -0.80874 -0.49736 -0.11817 -0.35128 -0.17627 -0.34755 -0.45423 -0.
 -0.15311  0.      -0.19883 -1.73001]
e : [-0.]
f : [-0.]
z : [-0.]
a : [-0.5]
b : [0.]
c : [-0.]
d : [0.]
q : [-0.86871 -2.40464  0.       0.      -0.44631 -1.54457 -2.69122  0.
 -0.19082 -0.42187 -0.21164 -1.52895 -0.24363 -0.62518 -0.45423 -0.
 -0.06026 -2.14994 -0.22416 -1.68457]
n : [0.25]
r : [ 0.       0.      -0.871   -0.88119 -2.31768 -2.14994 -0.      -1.29777
 -0.78543 -2.14994  0.       0.      -1.86867 -1.33957 -1.00992 -0.17077
 -2.14994 -1.2068  -2.14994 -1.125  ]


## **2. Enhancement:**

We add the teammate preferences/dispreferences matrices as new constraints.

### **a. Teammate Dispreferences:** 
Use the MD matrix to create groupings that ensure worker is placed in a project with fewer than [a number of] dispreferred teammates

(*) The straightforward optimal solution already satisfies <= 1 dispreferred teammate for each worker in every project (see below). We can enhance from there:
- try 0 (i.e. no [<= 0] dispreferred teammate): infeasible 

In [ ]:
(MD_df @ formatted_X) * formatted_X <= formatted_X * 1 # satisfies <= 1 

,Project 1,Project 2,Project 3,Project 4
Ash,True,True,True,True
Brock,True,True,True,True
Caitlin,True,True,True,True
Dahlia,True,True,True,True
Elaine,True,True,True,True
Flint,True,True,True,True
Gary,True,True,True,True
Hala,True,True,True,True
Iris,True,True,True,True
Jasmine,True,True,True,True


In [ ]:
(MD_df @ formatted_X) * formatted_X <= formatted_X * 0 # not satisfy <= 0 

,Project 1,Project 2,Project 3,Project 4
Ash,True,True,True,False
Brock,True,True,True,True
Caitlin,True,True,True,True
Dahlia,True,True,True,True
Elaine,True,False,True,True
Flint,True,True,True,True
Gary,True,True,True,True
Hala,True,True,True,True
Iris,True,True,True,True
Jasmine,True,True,True,True


In [ ]:
MD = MD_df.values
constraints_avoid_dispreferences_0 = constraints.copy()
constraints_avoid_dispreferences_0.extend([MD @ X <= max_number_of_dispreferred(0)]) 

problem = cp.Problem(objective, constraints_avoid_dispreferences_0)
try:
  problem.solve(qcp=True)
except KeyError:
  problem.solve(qcp=True, solver=cp.ECOS_BB)

print("status:", problem.status)
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", objective.value)
  print("resulting matrix:\n", X.value)
  print("formatted:")

formatted_X_MD = pd.DataFrame(np.round(np.abs(X.value), 0), index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8) if X.value is not None else None
formatted_X_MD

status: infeasible


### **b. Teammate Preferences:** 
Use the MP matrix to create groupings that ensure each worker is placed in a project with at least [a number of] preferred teammates

(*) The straightforward optimal solution fails at considering >= 1 preferred teammate for each worker in every project (see below). We can enhance from there:
- try 1 (i.e. >= 1 preferred teammate): optimal 
- try 2 (i.e. >= 2 preferred teammates): infeasible

In [ ]:
(MP_df @ formatted_X) * formatted_X >= formatted_X * 1 # not satisfy <= 1 

,Project 1,Project 2,Project 3,Project 4
Ash,True,True,True,True
Brock,True,True,True,True
Caitlin,True,True,True,True
Dahlia,True,True,True,True
Elaine,True,True,True,True
Flint,True,False,True,True
Gary,True,True,True,True
Hala,True,True,True,True
Iris,True,True,True,True
Jasmine,True,True,True,True


In [ ]:
MP = MP_df.values

at_least_number_of_preferred = X * 1

constraints_only_preferences_1 = constraints.copy()
constraints_only_preferences_1.extend([MP @ X >= at_least_number_of_preferred]) 

problem = cp.Problem(objective, constraints_only_preferences_1)
try:
  problem.solve(qcp=True)
except KeyError:
  problem.solve(qcp=True, solver=cp.ECOS_BB)

print("status:", problem.status)
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", objective.value)
  print("resulting matrix:\n", X.value)
  print("formatted:")

formatted_X_MP_1 = pd.DataFrame(np.round(np.abs(X.value), 0), index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8) if X.value is not None else None
formatted_X_MP_1

status: optimal
optimal value: 153.0
resulting matrix:
 [[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
formatted:


,Project 1,Project 2,Project 3,Project 4
Ash,0,0,0,1
Brock,0,0,0,1
Caitlin,1,0,0,0
Dahlia,1,0,0,0
Elaine,0,1,0,0
Flint,0,1,0,0
Gary,0,1,0,0
Hala,1,0,0,0
Iris,0,0,1,0
Jasmine,0,0,0,1


In [ ]:
MP = MP_df.values

at_least_number_of_preferred = X * 2

constraints_only_preferences_2 = constraints.copy()
constraints_only_preferences_2.extend([MP @ X >= 2]) 

problem = cp.Problem(objective, constraints_only_preferences_2)
try:
  problem.solve(qcp=True)
except KeyError:
  problem.solve(qcp=True, solver=cp.ECOS_BB)

print("status:", problem.status)
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", objective.value)
  print("resulting matrix:\n", X.value)
  print("formatted:")

formatted_X_MP_2 = pd.DataFrame(np.round(np.abs(X.value), 0), index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8) if X.value is not None else None
formatted_X_MP_2

status: infeasible


# **III. MINIMIZE SKILLS DIFFERENCES**

This section presents the second optimization approach: minimizing the total skills differences. In other words, we are minimizing the total absolute differences between the total skills of workers in each project. Not only does this optimization problem seeks to create a fair grouping.

We also present three differences approaches in modifying the constraints: the provided constraints, enhancement with teammate dispreferences, and enhancement with teammate preferences.

*For details regarding the optimization problem's formulation and motivation, please check our optimization project report*

## **1. Straightforward attempt:** 

We use the provided constraints. 

STATUS: optimal

In [ ]:
S_df.T @ formatted_X # result of S.T @ X

,Project 1,Project 2,Project 3,Project 4
Optimization,15,21,14,44
Organization,22,26,22,37
Teamwork,28,22,40,45


In [ ]:
sum_skills = (S_df.T @ formatted_X).sum()
sum_skills # the SS_i values 

Project 1     65
Project 2     69
Project 3     76
Project 4    126
dtype: int64

In [ ]:
diff = [abs(sum_skills[i] - sum_skills[j]) for i in range(4) for j in range(i+1, 4)]
diff_df = pd.DataFrame(data=diff, index=['P1,P2', 'P1,P3', 'P1,P4', 'P2,P3', 'P2,P4', 'P3,P4'], columns=['Difference'], dtype=np.int8)
print('Total differences:', np.sum(diff))
diff_df # how the objective function is derived

Total differences: 190


,Difference
"P1,P2",4
"P1,P3",11
"P1,P4",61
"P2,P3",7
"P2,P4",57
"P3,P4",50


In [ ]:
# DECLARE MAIN OBJECTIVE FUNCTION
S_sum = cp.sum((S.T @ X), axis=0) # sum of all skills per project, dim = (4,1)
D = [cp.abs(S_sum[i] - S_sum[j]) for i in range(4) for j in range(i+1, 4)] # 6 pairs of absolute differences in total members 

obj_function = cp.sum(D)
objective = cp.Minimize(obj_function)

In [ ]:
# CALL OPTIMIZATION AND GET RESULT
constraints = constraints_shared + constraints_1 + constraints_2 + constraints_3 + constraints_4

problem = cp.Problem(objective, constraints)
try:
  problem.solve(qcp=True)
except KeyError:
  problem.solve(qcp=True, solver=cp.ECOS_BB)

print("status:", problem.status)
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", objective.value)
  print("resulting matrix:\n", X.value)
  print("formatted:")

formatted_X = pd.DataFrame(np.round(np.abs(X.value), 0), index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8) if X.value is not None else None
formatted_X

status: optimal
optimal value: 0.0
resulting matrix:
 [[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
formatted:


,Project 1,Project 2,Project 3,Project 4
Ash,0,1,0,0
Brock,0,0,1,0
Caitlin,0,0,0,1
Dahlia,0,0,1,0
Elaine,1,0,0,0
Flint,0,0,1,0
Gary,0,0,1,0
Hala,0,0,1,0
Iris,0,1,0,0
Jasmine,0,0,0,1


## **2. Enhancement:**

We add the teammate preferences/dispreferences matrices as new constraints.

### **a. Teammate Dispreferences:** 
Use the MD matrix to create groupings that ensure worker is placed in a project with fewer than [a number of] dispreferred teammates

(*) The straightforward optimal solution already satisfies <= 2 dispreferred teammate for each worker in every project (see below). We can enhance from there:
- try 1 (i.e. no dispreferred teammate): optimal 
- try 0 (i.e. no dispreferred teammate): optimal 

In [ ]:
(MD_df @ formatted_X) * formatted_X <= formatted_X * 2 # satisfies <= 2

,Project 1,Project 2,Project 3,Project 4
Ash,True,True,True,True
Brock,True,True,True,True
Caitlin,True,True,True,True
Dahlia,True,True,True,True
Elaine,True,True,True,True
Flint,True,True,True,True
Gary,True,True,True,True
Hala,True,True,True,True
Iris,True,True,True,True
Jasmine,True,True,True,True


In [ ]:
(MD_df @ formatted_X) * formatted_X <= formatted_X * 1 # not satisfy <= 1 

,Project 1,Project 2,Project 3,Project 4
Ash,True,True,True,True
Brock,True,True,True,True
Caitlin,True,True,True,True
Dahlia,True,True,True,True
Elaine,False,True,True,True
Flint,True,True,True,True
Gary,True,True,True,True
Hala,True,True,True,True
Iris,True,True,True,True
Jasmine,True,True,True,True


In [ ]:
MD = MD_df.values

constraints_avoid_dispreferences_1 = constraints.copy()
constraints_avoid_dispreferences_1.extend([MD @ X <= max_number_of_dispreferred(1)]) 

problem = cp.Problem(objective, constraints_avoid_dispreferences_1)
try:
  problem.solve(qcp=True)
except KeyError:
  problem.solve(qcp=True, solver=cp.ECOS_BB)

print("status:", problem.status)
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", objective.value)
  print("resulting matrix:\n", X.value)
  print("formatted:")

formatted_X_MD_1 = pd.DataFrame(np.round(np.abs(X.value), 0), index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8) if X.value is not None else None
formatted_X_MD_1

status: optimal
optimal value: 0.0
resulting matrix:
 [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
formatted:


,Project 1,Project 2,Project 3,Project 4
Ash,0,0,0,1
Brock,0,0,1,0
Caitlin,0,0,0,1
Dahlia,0,0,1,0
Elaine,1,0,0,0
Flint,0,0,0,1
Gary,0,1,0,0
Hala,0,1,0,0
Iris,1,0,0,0
Jasmine,0,1,0,0


In [ ]:
MD = MD_df.values

constraints_avoid_dispreferences_0 = constraints.copy()
constraints_avoid_dispreferences_0.extend([MD @ X <= max_number_of_dispreferred(0)]) 

problem = cp.Problem(objective, constraints_avoid_dispreferences_0)
try:
  problem.solve(qcp=True)
except KeyError:
  problem.solve(qcp=True, solver=cp.ECOS_BB)

print("status:", problem.status)
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", objective.value)
  print("resulting matrix:\n", X.value)
  print("formatted:")

formatted_X_MD_0 = pd.DataFrame(np.round(np.abs(X.value), 0), index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8) if X.value is not None else None
formatted_X_MD_0

status: optimal
optimal value: 0.0
resulting matrix:
 [[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
formatted:


,Project 1,Project 2,Project 3,Project 4
Ash,0,1,0,0
Brock,0,0,1,0
Caitlin,1,0,0,0
Dahlia,1,0,0,0
Elaine,0,0,1,0
Flint,0,0,0,1
Gary,0,0,1,0
Hala,1,0,0,0
Iris,1,0,0,0
Jasmine,0,0,0,1


### **b. Teammate Preferences:** 
Use the MP matrix to create groupings that ensure each worker is placed in a project with at least [a number of] preferred teammates

(*) The straightforward optimal solution fails at considering >= 1 preferred teammate for each worker in every project (see below). We can enhance from there:
- try 1 (i.e. >= 1 preferred teammate): optimal 
- try 2 (i.e. >= 2 preferred teammates): infeasible

In [ ]:
(MP_df @ formatted_X) * formatted_X >= formatted_X * 1 # not satisfy <= 1 

,Project 1,Project 2,Project 3,Project 4
Ash,True,False,True,True
Brock,True,True,True,True
Caitlin,True,True,True,True
Dahlia,True,True,False,True
Elaine,True,True,True,True
Flint,True,True,True,True
Gary,True,True,True,True
Hala,True,True,True,True
Iris,True,True,True,True
Jasmine,True,True,True,False


In [ ]:
MP = MP_df.values

at_least_number_of_preferred = X * 1

constraints_only_preferences_1 = constraints.copy()
constraints_only_preferences_1.extend([MP @ X >= at_least_number_of_preferred]) 

problem = cp.Problem(objective, constraints_only_preferences_1)
problem.solve(qcp=True)

print("status:", problem.status)
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", objective.value)
  print("resulting matrix:\n", X.value)
  print("formatted:")

formatted_X_MP_1 = pd.DataFrame(np.round(np.abs(X.value), 0), index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8) if X.value is not None else None
formatted_X_MP_1

status: optimal
optimal value: 0.0
resulting matrix:
 [[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]
formatted:


,Project 1,Project 2,Project 3,Project 4
Ash,1,0,0,0
Brock,0,0,0,1
Caitlin,0,1,0,0
Dahlia,1,0,0,0
Elaine,0,0,1,0
Flint,1,0,0,0
Gary,0,0,0,1
Hala,0,0,0,1
Iris,1,0,0,0
Jasmine,0,0,1,0


In [ ]:
MP = MP_df.values

at_least_number_of_preferred = X * 2

constraints_only_preferences_2 = constraints.copy()
constraints_only_preferences_2.extend([MP @ X >= at_least_number_of_preferred]) 

problem = cp.Problem(objective, constraints_only_preferences_2)
try:
  problem.solve(qcp=True)
except KeyError:
  problem.solve(qcp=True, solver=cp.ECOS_BB)

print("status:", problem.status)
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
  print("optimal value:", objective.value)
  print("resulting matrix:\n", X.value)
  print("formatted:")

formatted_X_MP_2 = pd.DataFrame(np.round(np.abs(X.value), 0), index=df['Name'].values, columns=['Project 1', 'Project 2', 'Project 3', 'Project 4'], dtype=np.int8) if X.value is not None else None
formatted_X_MP_2

status: infeasible
